<a href="https://colab.research.google.com/github/ekaratnida/Applied-machine-learning/blob/master/Week04-workshop-1/Sklearn_to_ONNX_Conversion_HousePrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting scikit-learn models to ONNX for use in Android apps

* ✍️ [Read the blog](https://towardsdatascience.com/deploying-scikit-learn-models-in-android-apps-with-onnx-b3adabe16bab)
* 🌐 [Checkout the GitHub repo](https://github.com/shubham0204/Scikit_Learn_Android_Demo)
* 📁 [Download this notebook](https://github.com/shubham0204/Google_Colab_Notebooks/blob/main/Sklearn_to_ONNX_Conversion.ipynb)

### 1️⃣ Installing `onnxruntime` and `skl2onnx`

In [ ]:
!pip install onnxruntime skl2onnx

### 2️⃣ Building the scikit-learn model

In [7]:
import pandas as pd
import numpy as np

#data = pd.read_csv( 'score.csv' )
data = pd.read_csv('https://raw.githubusercontent.com/ekaratnida/Applied-machine-learning/master/Week04-workshop-1/data.txt')
data.head()

area  rooms   price
0  2104      3  399900
1  1600      3  329900
2  2400      3  369000
3  1416      2  232000
4  3000      4  539900

In [14]:
from sklearn.linear_model import LinearRegression

X , y = data.values[ : , 0:2 ] , data.values[ : , 2]
print('X = ',X)
print('y = ',y)
#X = np.expand_dims( X , axis=1 )
regressor = LinearRegression()
regressor.fit( X , y )

X =  [[2104    3]
 [1600    3]
 [2400    3]
 [1416    2]
 [3000    4]
 [1985    4]
 [1534    3]
 [1427    3]
 [1380    3]
 [1494    3]
 [1940    4]
 [2000    3]
 [1890    3]
 [4478    5]
 [1268    3]
 [2300    4]
 [1320    2]
 [1236    3]
 [2609    4]
 [3031    4]
 [1767    3]
 [1888    2]
 [1604    3]
 [1962    4]
 [3890    3]
 [1100    3]
 [1458    3]
 [2526    3]
 [2200    3]
 [2637    3]
 [1839    2]
 [1000    1]
 [2040    4]
 [3137    3]
 [1811    4]
 [1437    3]
 [1239    3]
 [2132    4]
 [4215    4]
 [2162    4]
 [1664    2]
 [2238    3]
 [2567    4]
 [1200    3]
 [ 852    2]
 [1852    4]
 [1203    3]]
y =  [399900 329900 369000 232000 539900 299900 314900 198999 212000 242500
 239999 347000 329999 699900 259900 449900 299900 199900 499998 599000
 252900 255000 242900 259900 573900 249900 464500 469000 475000 299900
 349900 169900 314900 579900 285900 249900 229900 345000 549000 287000
 368500 329900 314000 299000 179900 299900 239500]


LinearRegression()

In [3]:
from sklearn.linear_model import LinearRegression

X , y = data.values[ : , 0 ] , data.values[ : , 1 ]
X = np.expand_dims( X , axis=1 )

regressor = LinearRegression()
regressor.fit( X , y )

LinearRegression()

In [10]:
print( f'Prediction for area = 2000, rooms = 3 is {regressor.predict([[2000,3]])[0]}' )

Prediction for area = 2000, rooms = 3 is 341805.20024106506


### 3️⃣ Conversion to `.onnx`

In [16]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Specify an initial type for the model ( similar to input shape for the model )
initial_type = [
    ( 'input_area_and_rooms' , FloatTensorType( [None,2] ) )
]

# Write the ONNX model to disk
converted_model = convert_sklearn( regressor , initial_types=initial_type )
with open( "sklearn_model_house.onnx", "wb" ) as f:
    f.write( converted_model.SerializeToString() )

### 4️⃣ Conversion of `.onnx` to `.ort` ( Optional )

As per [Scott McKay's comment](https://github.com/shubham0204/Scikit_Learn_Android_Demo/issues/1),

> It’s not necessary to convert to .ort format when using the ‘full’ ONNX Runtime package onnxruntime-android - you can use the onnx model.
> The conversion to .ort format is only necessary if using the smaller ‘mobile’ package onnxruntime-mobile, which has limited operators/types (based on popular dnn models used in mobile scenarios) to provide a smaller binary size. That package however does not include traditional ML operators that SciKit-Learn tends to use, so most likely it wouldn't be able to run a model that was converted from SKL.


In [17]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort sklearn_model_house.onnx

Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /content/sklearn_model_house.onnx to ORT format model /content/sklearn_model_house.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2024-02-23 04:40:50,961 ort_format_model.utils [INFO] - Created config in /content/sklearn_model_house.required_operators.config
Converting models with optimization style 'Runtime' and level 'all'
Converting optimized ONNX model /content/sklearn_model_house.onnx to ORT format model /content/sklearn_model_house.with_runtime_opt.ort
Converted 1/1 models successfully.
Converting models again without runtime optimizations to generate a complete config file. These converted models are temporary and will be deleted.
Converting optimized ONNX model /content/sklearn_model_house.onnx to ORT format model /content/tmpymmfj3gn.without_runtime_opt/sklearn_model_house.ort
Converted 1/1 model